<a href="https://colab.research.google.com/github/AsmaaYassinDev/Behavioural-Anomaly-Detection-for-ATO-Fraud/blob/main/Unsupervised_LOF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

print("--- Execution Started (Improved LOF) ---")
print("Objective: Use LOF with full behavioral + engineered features.")

# --- Load Data ---
file_path = '/content/drive/My Drive/Colab_Data/PS_20174392719_1491204439457_log.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded the full file ({len(df)} rows).")
except Exception as e:
    print(f"Error during data loading: {e}")
    exit()

# --- Step 1: Create the 'Smart Sample' ---
print("\n--- Step 1: Creating the 'Smart Sample' ---")
df_fraud = df[df['isFraud'] == 1]
fraud_dest_ids = df_fraud['nameDest'].unique()
fraud_orig_ids = df_fraud['nameOrig'].unique()
all_fraud_user_ids = np.union1d(fraud_dest_ids, fraud_orig_ids)
df_fraud_lifecycle = df[
    df['nameOrig'].isin(all_fraud_user_ids) |
    df['nameDest'].isin(all_fraud_user_ids)
]
df_normal = df[df['isFraud'] == 0]
sample_size = min(500000, len(df_normal))
df_normal_sample = df_normal.sample(n=sample_size, random_state=42)
df_smart_sample = pd.concat([df_fraud_lifecycle, df_normal_sample]).drop_duplicates(keep='first')
print(f"The final 'Smart Sample' was created with {len(df_smart_sample)} rows.")

# --- Step 2: Build Behavioral Profiles (Strong Features) ---
print("\n--- Step 2: Building Behavioral Profiles ---")

# (a) Calculate basic statistics
df_received = df_smart_sample[df_smart_sample['type'].isin(['TRANSFER', 'CASH_IN'])]
total_received = df_received.groupby('nameDest')['amount'].sum().to_dict()
unique_senders = df_received.groupby('nameDest')['nameOrig'].nunique().to_dict()
df_cashed_out = df_smart_sample[df_smart_sample['type'] == 'CASH_OUT']
total_cashed_out = df_cashed_out.groupby('nameOrig')['amount'].sum().to_dict()

all_user_ids = set(total_received.keys()) | set(total_cashed_out.keys()) | set(unique_senders.keys())
profiles_list = []
for user_id in all_user_ids:
    received = total_received.get(user_id, 0)
    cashed_out = total_cashed_out.get(user_id, 0)
    senders = unique_senders.get(user_id, 0)
    ratio = (cashed_out / (received + 1e-6))
    ratio = min(ratio, 1.0)
    profiles_list.append({
        'user_id': user_id,
        'dest_cash_out_ratio': ratio,
        'dest_unique_senders': senders
    })
final_profiles = pd.DataFrame(profiles_list)
print("Behavioral profiles created successfully.")

# --- Step 3: Merge Features with Transactions ---
print("\n--- Step 3: Merging Features with Transactions ---")
df_model_data = pd.merge(df_smart_sample, final_profiles, left_on='nameDest', right_on='user_id', how='left')
df_model_data = pd.merge(df_model_data, final_profiles, left_on='nameOrig', right_on='user_id', how='left', suffixes=('_dest', '_orig'))

df_model_data['dest_cash_out_ratio_dest'] = df_model_data['dest_cash_out_ratio_dest'].fillna(0)
df_model_data['dest_unique_senders_dest'] = df_model_data['dest_unique_senders_dest'].fillna(0)
df_model_data['dest_cash_out_ratio_orig'] = df_model_data['dest_cash_out_ratio_orig'].fillna(0)
df_model_data['dest_unique_senders_orig'] = df_model_data['dest_unique_senders_orig'].fillna(0)

# --- Step 4: Train Unsupervised LOF Model ---
print("\n--- Step 4: Training Local Outlier Factor (LOF) Model ---")

features = [
    'amount',
    'dest_cash_out_ratio_dest',
    'dest_unique_senders_dest',
    'dest_cash_out_ratio_orig',
    'dest_unique_senders_orig'
]
df_model_data['type_encoded'] = df_model_data['type'].astype('category').cat.codes
features.append('type_encoded')

X = df_model_data[features]
y_true = df_model_data['isFraud'] # The "Correct Answer" (for comparison only)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

contamination = y_true.mean()
print(f"Fraud (Contamination) rate in the Smart Sample: {contamination:.2%}")

# --- (This is the change) ---
# n_neighbors=20 is a common default setting
model = LocalOutlierFactor(n_neighbors=20, contamination=contamination)

# LOF uses .fit_predict() to give us -1 (anomalous) or 1 (normal)
predictions = model.fit_predict(X_scaled)
print("Model training complete.")

# --- Step 5: Evaluation (Comparison) ---
print("\n--- Step 5: Evaluating the LOF Model ---")

# Convert the model's "guess" (-1) to (1) to match 'isFraud'
y_pred = [1 if p == -1 else 0 for p in predictions]

f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score (Final score for LOF): {f1:.2%}")
print("\nConfusion Matrix:")
print(cm)

print("\n--- LOF Code Execution Complete ---")

--- Execution Started (Improved LOF) ---
Objective: Use LOF with full behavioral + engineered features.
Successfully loaded the full file (6362620 rows).

--- Step 1: Creating the 'Smart Sample' ---
The final 'Smart Sample' was created with 561154 rows.

--- Step 2: Building Behavioral Profiles ---
Behavioral profiles created successfully.

--- Step 3: Merging Features with Transactions ---

--- Step 4: Training Local Outlier Factor (LOF) Model ---
Fraud (Contamination) rate in the Smart Sample: 1.46%
Model training complete.

--- Step 5: Evaluating the LOF Model ---
Precision: 4.94%
Recall: 4.94%
F1-Score (Final score for LOF): 4.94%

Confusion Matrix:
[[545134   7807]
 [  7807    406]]

--- LOF Code Execution Complete ---
